# Model Development as Described in "Risk Stratification with Explainable Machine Learning for 30-Day Procedure-Related Mortality and 30-Day Unplanned Readmission in Patients with Peripheral Arterial Disease"

Meredith Cox; J.C. Panagides; Azadeh Tabari, MD; Sanjeeva Kalva, MD; Jayashree Kalpathy-Cramer, PhD; Dania Daye, MD, PhD

## Imports

In [ ]:
import copy
import joblib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymrmr
import shap

from imblearn.over_sampling import SMOTE, ADASYN
from lifelines import KaplanMeierFitter, CoxPHFitter, statistics
from sklearn import metrics
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, average_precision_score, brier_score_loss, confusion_matrix, classification_report, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV

# Model Development

## Defining useful methods

In [ ]:
def get_confidence_interval(y_pred,y_true):
    '''
    Calculates the confidence interval of the AUC score using bootstrapping
    Code adapted from: https://stackoverflow.com/questions/19124239/scikit-learn-roc-curve-with-confidence-intervals
    '''
    n_bootstraps = 1000
    rng_seed = 0
    bootstrapped_scores = []
    
    y_true.reset_index(drop=True,inplace=True)

    rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        indices = rng.randint(0, len(y_pred), len(y_pred))
        if len(np.unique(y_true[indices])) < 2:
            continue

        score = roc_auc_score(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    confidence_lower = sorted_scores[int(0.05 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.95 * len(sorted_scores))]
    print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
        confidence_lower, confidence_upper))
    return confidence_lower, confidence_upper

In [ ]:
def evaluate(X_test,y_test,model):
    '''
    Calculates metrics for evaluation: Accuracy, AUC, AU-PRC, 
    and shows confusion matric and classification report including
    sensitivity and specificity.
    '''
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    auc = roc_auc_score(y_test, probs)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)
    pr = average_precision_score(y_test, probs)

    print("Accuracy: " + str(clf_rf.score(X_test, y_test)))
    print("AUC: " + str(auc))
    print("AUPRC: " + str(pr) + '\n')

    # Getting the decision threshold
    # Code from here: https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/

    yhat = clf_rf.predict_proba(X_test)
    yhat = yhat[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, yhat)
    J = tpr - fpr
    ix = np.argmax(J)
    best_thresh = thresholds[ix]

    print('Best Threshold=%f' % (best_thresh))

    predicts = (clf_rf.predict_proba(X_test)[:,1] >= best_thresh).astype(int)

    print(confusion_matrix(y_test, predicts))
    print(classification_report(y_test, predicts, target_names=['0','1']))

    get_confidence_interval(probs,y_test)
    
    return fpr, tpr, auc, probs


## Training the model

### Death

In [ ]:
df_train = pd.read_csv("NSQIP_LEE_2008-2018_optimpute_train_mortality_readmission.csv")
df_val = pd.read_csv("NSQIP_LEE_2008-2018_optimpute_val_mortality_readmission.csv")
df_test = pd.read_csv("NSQIP_LEE_2008-2018_optimpute_test_mortality_readmission.csv")

outcome_colnames = ['LEE_ULP', 'LEE_BLEEDING','LEE_MI_STROKE','SUPINFEC','WNDINFD','ORGSPCSSI',
                    'DEHIS','OUPNEUMO','REINTUB','PULEMBOL','FAILWEAN','RENAINSF','OPRENAFL',
                    'URNINFEC','CNSCVA','CDARREST','CDMI','OTHBLEED','OTHDVT','OTHSYSEP',
                    'OTHSESHOCK','THROMBOSIS','AMPUTATION','DEATH']
outcomes_train = df_train[outcome_colnames]
outcomes_val = df_val[outcome_colnames]
outcomes_test = df_test[outcome_colnames]

predictors_train = df_train.drop(columns=outcome_colnames)
predictors_train = predictors_train.drop(columns=['LEE_AMPUTATION','LEE_WOUND'])

predictors_val = df_val.drop(columns=outcome_colnames)
predictors_val = predictors_val.drop(columns=['LEE_AMPUTATION','LEE_WOUND'])

predictors_test = df_test.drop(columns=outcome_colnames)
predictors_test = predictors_test.drop(columns=['LEE_AMPUTATION','LEE_WOUND'])

outcomes_train.DEATH = outcomes_train.DEATH.round()
outcomes_val.DEATH = outcomes_val.DEATH.round()
outcomes_test.DEATH = outcomes_test.DEATH.round()

In [ ]:
outcome_to_test = "DEATH" #CHANGE TO THE OUTCOME YOU WANT TO TEST

selected_features = ['PRPLATE',
 'renalcomorb',
 'fnstatus',
 'dyspnea',
 'Symptoms_Claudication',
 'PRBILI',
 'LEE_HRF_PHYS',
 'pulmcomorb',
 'PRALBUM',
 'ELECTSURG',
 'PRBUN',
 'PRWBC',
 'PRSGOT',
 'PRALKPH',
 'Symptoms_Asymptomatic',
 'diabetes',
 'PRCREAT',
 'PRHCT',
 'PRINR',
 'PRPTT'] #Insert selected features from preprocessing step

X_train = predictors_train[selected_features]
X_val = predictors_val
X_test = predictors_test[selected_features]

y_train = outcomes_train[outcome_to_test]
y_val = outcomes_val[outcome_to_test]
y_test = outcomes_test[outcome_to_test]

#Oversample
oversample = ADASYN()
X, y = oversample.fit_resample(X_train, y_train)

print(outcome_to_test)

# Initialize the random forest with default params, will tune later
clf_rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1,max_depth=4, max_features='auto', bootstrap=True, random_state=0)
# Fit the model
clf_rf.fit(X, y)

In [ ]:
# Evaluation of the initial model
fpr, tpr, auc, probs = evaluate (X_test,y_test, clf_rf)

In [ ]:
# Training set evaluation
probs_train = clf_rf.predict_proba(X_train)
probs_train = probs_train[:, 1]
auc = roc_auc_score(y_train, probs_train)
print("Accuracy: " + str(clf_rf.score(X_train, y_train)))
fpr, tpr, thresholds = metrics.roc_curve(y_train, probs_train)
print("AUC: " + str(metrics.auc(fpr, tpr)))
pr = average_precision_score(y_train, probs_train)
print("AUPRC: " + str(pr) + '\n')

predicts_train = (clf_rf.predict_proba(X_train)[:,1] >= .5).astype(int)

target_names = ['0','1']
print(classification_report(y_train, predicts_train, target_names=target_names))
confusion_matrix(y_train, predicts_train)

### Tuning with grid search

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'None','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 7, num = 4)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

def auc_scorer(ground_truth, predictions):
    fpr, tpr, _ = roc_curve(ground_truth, predictions[:, 1], pos_label=1)    
    return auc(fpr, tpr)

my_auc_scorer = make_scorer(auc_scorer, greater_is_better=True, needs_proba=True)

param_grid

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 2,scoring = my_auc_scorer)
grid_search.fit(X, y)
grid_search.best_params_

In [ ]:
# The optimal params were the same as the default, so we don't need to evaluate a new model

### Plotting

In [ ]:
#Plotting ROC curve of the best model (random forest)

fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)
roc_auc = metrics.auc(fpr, tpr)

plt.figure(dpi=300)
plt.title('Receiver Operating Characteristic: 30-day Mortality')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Model interpretation

### Gini Importance

In [ ]:
importances = clf_rf.feature_importances_
indices = np.argsort(importances)
features = X_train.columns
print([features[i] for i in indices])
print(importances[indices])
plt.rcParams['figure.dpi'] = 300
plt.figure(figsize = ( 7 , 7 ))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')

plt.yticks(range(len(indices)),[features[i] for i in indices])
plt.xlabel('Relative Importance',fontsize=14)
plt.tick_params(
    axis='y',          
    which='both',      
    left=False,
    labelsize=14) 

plt.show()

### SHAP

In [ ]:
explainer = shap.KernelExplainer(clf_rf.predict,X_test)
rf_shap_values = explainer.shap_values(X_test,nsamples=100)
shap.summary_plot(rf_shap_values, X_test.iloc[:5037,:])

## Fairness Analysis

In [ ]:
def fairness_analysis(all_features_test, y_test, clf_rf, predictors):

    X_test_fairness = all_features_test
    features_fairness = list(X_test.columns)
    features_fairness.extend(['Race_white','SEX','AGE'])
    X_test_fairness = X_test_fairness[features_fairness]

    y_test_fairness = y_test

    X_test_white = X_test_fairness.loc[X_test_fairness['Race_white'] == 1]
    X_test_nonwhite = X_test_fairness.loc[X_test_fairness['Race_white'] != 1]

    X_test_white = X_test_white.drop(columns=['Race_white','SEX','AGE'])
    X_test_nonwhite = X_test_nonwhite.drop(columns=['Race_white','SEX','AGE'])

    y_test_white = y_test_fairness.loc[X_test_fairness['Race_white'] == 1]
    y_test_nonwhite = y_test_fairness.loc[X_test_fairness['Race_white'] != 1]

    X_test_female = X_test_fairness.loc[X_test_fairness['SEX'] == 0]
    X_test_male = X_test_fairness.loc[X_test_fairness['SEX'] == 1]

    X_test_female = X_test_female.drop(columns=['Race_white','SEX','AGE'])
    X_test_male = X_test_male.drop(columns=['Race_white','SEX','AGE'])

    y_test_female = y_test_fairness.loc[X_test_fairness['SEX'] == 0]
    y_test_male = y_test_fairness.loc[X_test_fairness['SEX'] == 1]

    X_test_over_65 = X_test_fairness.loc[X_test_fairness['AGE'] > 65]
    X_test_under_65 = X_test_fairness.loc[X_test_fairness['AGE'] <= 65]

    X_test_over_65 = X_test_over_65.drop(columns=['Race_white','SEX','AGE'])
    X_test_under_65 = X_test_under_65.drop(columns=['Race_white','SEX','AGE'])

    y_test_over_65 = y_test_fairness.loc[X_test_fairness['AGE'] > 65]
    y_test_under_65 = y_test_fairness.loc[X_test_fairness['AGE'] <= 65]


    print("White")
    fpr_white, tpr_white, roc_auc_white, preds_white = evaluate(X_test_white,y_test_white,clf_rf)
    get_confidence_interval(preds_white,y_test_white)
    print("Nonwhite")
    fpr_nonwhite, tpr_nonwhite, roc_auc_nonwhite, preds_nonwhite = evaluate(X_test_nonwhite,y_test_nonwhite,clf_rf)
    get_confidence_interval(preds_nonwhite,y_test_nonwhite)
    print("Male")
    fpr_male, tpr_male, roc_auc_male, preds_male = evaluate(X_test_male,y_test_male,clf_rf)
    get_confidence_interval(preds_male,y_test_male)
    print("Female")
    fpr_female, tpr_female, roc_auc_female, preds_female = evaluate(X_test_female,y_test_female,clf_rf)
    get_confidence_interval(preds_female,y_test_female)
    print("Over 65")
    fpr_over_65, tpr_over_65, roc_auc_over_65, preds_over_65 = evaluate(X_test_over_65,y_test_over_65,clf_rf)
    get_confidence_interval(preds_over_65,y_test_over_65)
    print("Under 65")
    fpr_under_65, tpr_under_65, roc_auc_under_65, preds_under_65 = evaluate(X_test_under_65,y_test_under_65,clf_rf)
    get_confidence_interval(preds_under_65,y_test_under_65)

    fig, axs = plt.subplots(3)
    fig.set_dpi(300)

    plt.rc('font', family='arial')
    axs[0].plot(fpr_white, tpr_white, 'darkblue', label = 'White' % roc_auc_white, linewidth=2)
    axs[0].plot(fpr_nonwhite, tpr_nonwhite, 'deepskyblue', label = 'Non-white' % roc_auc_nonwhite, linewidth=2)
    axs[0].plot([0, 1], [0, 1],'--',color='red',label='No Skill (Chance)',alpha=.9)
    axs[0].set_title("White vs. Non-white", fontsize=11)
    axs[0].legend(loc = 'lower right')
    axs[0].set_xlim([0, 1])
    axs[0].set_ylim([0, 1])
    axs[0].set_ylabel('Sensitivity')
    axs[0].set_xlabel('Specificity')
    axs[0].grid(True,which='major',axis='both',alpha=0.2)

    axs[1].plot(fpr_male, tpr_male, 'darkgreen', label = 'Male' % roc_auc_male, linewidth=2)
    axs[1].plot(fpr_female, tpr_female, 'lightgreen', label = 'Female' % roc_auc_female, linewidth=2)
    axs[1].plot([0, 1], [0, 1],'--',color='red',label='No Skill (Chance)',alpha=.9)
    axs[1].set_title("Male vs. Female", fontsize=11)
    axs[1].legend(loc = 'lower right')
    axs[1].set_xlim([0, 1])
    axs[1].set_ylim([0, 1])
    axs[1].set_ylabel('Sensitivity')
    axs[1].set_xlabel('Specificity')
    axs[1].grid(True,which='major',axis='both',alpha=0.2)

    axs[2].plot(fpr_over_65, tpr_over_65, 'indigo', label = 'Age >= 65' % roc_auc_over_65, linewidth=2)
    axs[2].plot(fpr_under_65, tpr_under_65, 'plum', label = 'Age < 65' % roc_auc_under_65, linewidth=2)
    axs[2].plot([0, 1], [0, 1],'--',color='red',label='No Skill (Chance)',alpha=.9)
    axs[2].set_title("Age 65 and Over vs. Under Age 65", fontsize=11)
    axs[2].legend(loc = 'lower right')
    axs[2].set_xlim([0, 1])
    axs[2].set_ylim([0, 1])
    axs[2].set_ylabel('Sensitivity')
    axs[2].set_xlabel('Specificity')
    axs[2].grid(True,which='major',axis='both',alpha=0.2)

    fig.set_figheight(10)
    fig.set_figwidth(5)
    fig.tight_layout()
    
    return y_test_white, y_test_nonwhite, preds_white, preds_nonwhite, y_test_male, y_test_female, preds_male, preds_female, y_test_over_65, y_test_under_65, preds_over_65, preds_under_65

y_test_white, y_test_nonwhite, preds_white, preds_nonwhite, y_test_male, y_test_female, preds_male, preds_female, y_test_over_65, y_test_under_65, preds_over_65, preds_under_65 = fairness_analysis(predictors_test,y_test,clf_rf,selected_features)


### Delong's Test for AUC curve difference

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i y_test_white -i y_test_nonwhite -i preds_white -i preds_nonwhite

install.packages("pROC")
library(pROC)

responsea<- y_test_white
responseb<- y_test_nonwhite
modela<- preds_white
modelb<- preds_nonwhite
roca<-roc(responsea,modela)
rocb<-roc(responseb,modelb)
roc.test(roca,rocb,method=c('delong'))

In [ ]:
%%R -i y_test_male -i y_test_female -i preds_male -i preds_female

install.packages("pROC")
library(pROC)

responsea<- y_test_male
responseb<- y_test_female
modela<- preds_male
modelb<- preds_female
roca<-roc(responsea,modela)
rocb<-roc(responseb,modelb)
roc.test(roca,rocb,method=c('delong'))

In [ ]:
%%R -i y_test_over_65 -i y_test_under_65 -i preds_over_65 -i preds_under_65

install.packages("pROC")
library(pROC)

responsea<- y_test_over_65
responseb<- y_test_under_65
modela<- preds_over_65
modelb<- preds_under_65
roca<-roc(responsea,modela)
rocb<-roc(responseb,modelb)
roc.test(roca,rocb,method=c('delong'))

### Readmission

In [ ]:
survival = pd.read_csv('survival.csv',index_col='Unnamed: 0')
survival.unplanned_readmission = survival.unplanned_readmission.replace(np.nan,0)
outcomes_train['unplanned_readmission'] = survival.unplanned_readmission[:7429] #2011-2015
outcomes_val['unplanned_readmission'] = survival.unplanned_readmission[7429:9407] #2016
outcomes_test['unplanned_readmission'] = survival.unplanned_readmission[9407:14444].reset_index(drop=True) #2017-2018

outcome_to_test = 'unplanned_readmission'

selected_features = ['PRPLATE',
 'MRTAS',
 'renalcomorb',
 'Symptoms_Claudication',
 'diabetes',
 'steroid',
 'fnstatus',
 'wndinf',
 'ASA3',
 'wtloss',
 'PRINR',
 'PRALBUM',
 'PRCREAT',
 'Symptoms_Asymptomatic',
 'PRBILI',
 'PRSGOT',
 'ELECTSURG',
 'WND3',
 'PRHCT']

X_train = predictors_train[selected_features]
X_val = predictors_val
X_test = predictors_test[selected_features]

y_train = outcomes_train[outcome_to_test]
y_val = outcomes_val[outcome_to_test]
y_test = outcomes_test[outcome_to_test]

#Oversample
oversample = ADASYN()
X, y = oversample.fit_resample(X_train, y_train)

print(outcome_to_test)

# Initialize the random forest with default params, will tune later
clf_rf = RandomForestClassifier(n_estimators=100, min_samples_split=2, min_samples_leaf=1,max_depth=4, max_features='auto', bootstrap=True, random_state=0)
# Fit the model
clf_rf.fit(X, y)

In [ ]:
fpr, tpr, auc, probs = evaluate (X_test, y_test, clf_rf)

In [ ]:
# Training set evaluation
probs_train = clf_rf.predict_proba(X_train)
probs_train = probs_train[:, 1]
auc = roc_auc_score(y_train, probs_train)
print("Accuracy: " + str(clf_rf.score(X_train, y_train)))
fpr, tpr, thresholds = metrics.roc_curve(y_train, probs_train)
print("AUC: " + str(metrics.auc(fpr, tpr)))
pr = average_precision_score(y_train, probs_train)
print("AUPRC: " + str(pr) + '\n')

predicts_train = (clf_rf.predict_proba(X_train)[:,1] >= .5).astype(int)

target_names = ['0','1']
print(classification_report(y_train, predicts_train, target_names=target_names))
confusion_matrix(y_train, predicts_train)

### Tuning with Grid Search

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'None','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(4, 7, num = 4)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

def auc_scorer(ground_truth, predictions):
    fpr, tpr, _ = roc_curve(ground_truth, predictions[:, 1], pos_label=1)    
    return auc(fpr, tpr)

my_auc_scorer = make_scorer(auc_scorer, greater_is_better=True, needs_proba=True)

param_grid

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 2,scoring = my_auc_scorer)
grid_search.fit(X, y)
grid_search.best_params_

In [ ]:
# The optimal params were the same as the default, so we don't need to evaluate a new model

### Plotting

In [ ]:
#Plotting ROC curve of the best model (random forest)

fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)
roc_auc = metrics.auc(fpr, tpr)

plt.figure(dpi=300)
plt.title('Receiver Operating Characteristic: 30-day Mortality')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Model interpretation

### Gini Importance

In [ ]:
importances = clf_rf.feature_importances_
indices = np.argsort(importances)
features = X_train.columns
print([features[i] for i in indices])
print(importances[indices])
plt.rcParams['figure.dpi'] = 300
plt.figure(figsize = ( 7 , 7 ))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')

plt.yticks(range(len(indices)),[features[i] for i in indices])
plt.xlabel('Relative Importance',fontsize=14)
plt.tick_params(
    axis='y',          
    which='both',      
    left=False,
    labelsize=14) 

plt.show()

### SHAP

In [ ]:
explainer = shap.KernelExplainer(clf_rf.predict,X_test)
rf_shap_values = explainer.shap_values(X_test,nsamples=100)
shap.summary_plot(rf_shap_values, X_test.iloc[:5037,:])

## Fairness Analysis

In [ ]:
y_test_white, y_test_nonwhite, preds_white, preds_nonwhite, y_test_male, y_test_female, preds_male, preds_female, y_test_over_65, y_test_under_65, preds_over_65, preds_under_65 = fairness_analysis(predictors_test,y_test,clf_rf,selected_features)


### Delong's Test for AUC Curve Difference

In [ ]:
%%R -i y_test_white -i y_test_nonwhite -i preds_white -i preds_nonwhite

install.packages("pROC")
library(pROC)

responsea<- y_test_white
responseb<- y_test_nonwhite
modela<- preds_white
modelb<- preds_nonwhite
roca<-roc(responsea,modela)
rocb<-roc(responseb,modelb)
roc.test(roca,rocb,method=c('delong'))

In [ ]:
%%R -i y_test_male -i y_test_female -i preds_male -i preds_female

install.packages("pROC")
library(pROC)

responsea<- y_test_male
responseb<- y_test_female
modela<- preds_male
modelb<- preds_female
roca<-roc(responsea,modela)
rocb<-roc(responseb,modelb)
roc.test(roca,rocb,method=c('delong'))

In [ ]:
%%R -i y_test_over_65 -i y_test_under_65 -i preds_over_65 -i preds_under_65

install.packages("pROC")
library(pROC)

responsea<- y_test_over_65
responseb<- y_test_under_65
modela<- preds_over_65
modelb<- preds_under_65
roca<-roc(responsea,modela)
rocb<-roc(responseb,modelb)
roc.test(roca,rocb,method=c('delong'))

# Survival Analysis

## Defining Functions

In [ ]:
def survival_binary(feature_name,outcome_name,survival_name,labels, title):
    '''
    Determining differences in survival for binary features (ones that do not require definition of a threshold)
    '''

    feature_all=pd.concat([predictors_train, predictors_val, predictors_test], axis=0)[feature_name].round().reset_index(drop=True)
    
    survival_1 = survival.loc[feature_all == 1]
    died_1 = outcomes_all.loc[feature_all == 1][outcome_name]
    survival_0 = survival.loc[feature_all != 1]
    died_0 = outcomes_all.loc[feature_all != 1][outcome_name]

    kmf_1 = KaplanMeierFitter()
    X_1 = survival_1[survival_name]
    Y_1 = died_1

    kmf_0 = KaplanMeierFitter()
    X_0 = survival_0[survival_name]
    Y_0 = died_0

    kmf_1.fit_right_censoring(X_1, event_observed = Y_1)
    kmf_0.fit_right_censoring(X_0, event_observed = Y_0)
    plt.figure(dpi=300)
    plt.ylabel("Survival")
    kmf_1.plot(label=labels[0])
    kmf_0.plot(label=labels[1])
    plt.title(title)
    plt.xlabel("Days after procedure")
    
    results = statistics.logrank_test(X_1, X_0, event_observed_A=Y_1, event_observed_B=Y_0)
    print("Log-rank test p-value: ", results.p_value) 
    

In [ ]:
def survival_continuous(feature_name,outcome_name,survival_name,labels, title, threshold):
    '''
    Determining differences in survival for continuous features (ones that require definition of a threshold)
    '''

    feature_all=pd.concat([predictors_train, predictors_val, predictors_test], axis=0)[feature_name].reset_index(drop=True)

    survival_0 = survival.loc[feature_all < threshold] 
    died_0 = outcomes_all.loc[feature_all < threshold][outcome_name]
    survival_1 = survival.loc[feature_all >= threshold]
    died_1 = outcomes_all.loc[feature_all >= threshold][outcome_name]

    kmf_0 = KaplanMeierFitter()
    X_0 = survival_0[survival_name]
    Y_0 = died_0

    kmf_1 = KaplanMeierFitter()
    X_1 = survival_1[survival_name]
    Y_1 = died_1

    plt.figure(dpi=300)
    kmf_0.fit_right_censoring(X_0, event_observed = Y_0)
    kmf_1.fit_right_censoring(X_1, event_observed = Y_1)
    plt.ylabel("Survival")
    plt.title(title)
    kmf_0.plot(label=str(feature_name+' < '+ str(threshold)))
    kmf_1.plot(label=str(feature_name+' >= '+ str(threshold)))
    plt.xlabel("Days after procedure")

    results = statistics.logrank_test(X_1, X_0, event_observed_A=Y_1, event_observed_B=Y_0)
    print("Log-rank test p-value: ", results.p_value) 

## Death

In [ ]:
survival.DOPERTOD = survival.DOPERTOD.replace(-99.0,30) #Convert -99 to 30 since we are looking at 30-day outcomes
survival['dead'] = survival.DOPERTOD.notnull()
outcomes_all = pd.concat([outcomes_train, outcomes_val, outcomes_test], axis=0).reset_index(drop=True)

In [ ]:
kmf = KaplanMeierFitter()
X= survival['DOPERTOD']
Y = outcomes_all.DEATH

kmf.fit_right_censoring(X, event_observed = Y)
kmf.plot()
plt.title("Kaplan Meier estimates: 30-day Mortality")
plt.xlabel("Days after procedure")
plt.ylabel("Survival")
plt.show()

In [ ]:
# Plotting survival curves between two groups

survival_binary('ELECTSURG','DEATH','DOPERTOD',('Elective Surgery','Nonelective Surgery'),'Elective Surgery')
survival_binary('LEE_HRF_PHYS','DEATH','DOPERTOD',('High Risk Factors','No High Risk Factors'),'Physiologic High Risk Factors')
survival_binary('fnstatus','DEATH','DOPERTOD',('Independent','Totally or Partially Dependent'),'Functional Status')
survival_continuous('PRHCT','DEATH','DOPERTOD',('HCT < 30','HCT >= 30'),'Hematocrit',30)
survival_continuous('PRCREAT','DEATH','DOPERTOD',('Creatinine < 30','Creatinine >= 30'),'Creatinine',1.3)

## Readmission

In [ ]:
survival.READMPODAYS1 = survival.READMPODAYS1.replace(-99.0,np.nan)
survival.READMPODAYS1 = survival.READMPODAYS1.replace(np.nan,30)
survival.unplanned_readmission = survival.unplanned_readmission.replace(np.nan,0)

outcomes_all['unplanned_readmission'] = survival.unplanned_readmission

In [ ]:
kmf3 = KaplanMeierFitter()
X= survival['READMPODAYS1']
Y= survival.unplanned_readmission

kmf3.fit_right_censoring(X, event_observed = Y)
kmf3.plot()
plt.title("Kaplan Meier estimates: 30-day Readmission")
plt.xlabel("Days after procedure")
plt.ylabel("Survival")
plt.show()

In [ ]:
# Plotting survival curves between two groups

survival_binary('wndinf','unplanned_readmission','READMPODAYS1',('Open Wound/Wound Infection','No Open Wound/Wound Infection'),'Open Wound/Wound Infection')
survival_binary('ELECTSURG','unplanned_readmission','READMPODAYS1',('Elective Surgery','Nonelective Surgery'),'Elective Surgery')
survival_binary('diabetes','unplanned_readmission','READMPODAYS1',('Diabetic','Non-diabetic'),'Diabetes')
survival_binary('Symptoms_Claudication','unplanned_readmission','READMPODAYS1',('Claudication','No Claudication'),'Claudication')
survival_death_binary('MRTAS','unplanned_readmission','READMPODAYS1',('MRTAS','No MRTAS'),'Major Reintervention of Treated Arterial Segment (MRTAS)')